In [2]:
import glob

import datetime

from IPython.display import display

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [6]:
val = pd.read_csv('../datasets/validation/validation.csv')
att = val['attack']
val.describe()

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C78,C79,C80,C81,C82,C83,C84,C85,C86,attack
count,86400.000000,86400.0,86400.000000,86400.000000,86400.000000,86400.000000,86400.000000,86400.0,86400.0,86400.0,...,86400.000000,86400.000000,86400.000000,86400.000000,86400.0,86400.000000,86400.000000,86400.0,86400.000000,86400.000000
mean,-2.459467,0.0,11.935542,-0.000027,15.921492,0.170474,12.743440,70.0,1.0,50.0,...,31.548518,0.716042,684.234297,365.849157,1.0,1004.488464,12.001176,50.0,1116.742674,0.010243
std,0.348438,0.0,0.224093,0.001636,10.159002,0.195639,0.108907,0.0,0.0,0.0,...,0.339997,0.000000,667.094271,59.099704,0.0,27.234351,0.057203,0.0,1695.167901,0.100689
min,-3.294500,0.0,11.766050,-0.012510,0.306700,-0.501600,12.669310,70.0,1.0,50.0,...,30.602420,0.716042,11.122750,235.659000,1.0,954.837890,12.000000,50.0,5.000000,0.000000
25%,-2.771800,0.0,11.788940,-0.001090,12.696840,0.032100,12.669310,70.0,1.0,50.0,...,31.352770,0.716042,277.271270,318.920040,1.0,985.227720,12.000000,50.0,7.000000,0.000000
50%,-2.454600,0.0,11.788940,-0.000070,14.581300,0.177600,12.669310,70.0,1.0,50.0,...,31.532440,0.716042,505.682950,365.704590,1.0,995.805910,12.000000,50.0,85.000000,0.000000
75%,-2.137600,0.0,12.261960,0.001010,16.229250,0.334800,12.903438,70.0,1.0,50.0,...,31.729280,0.716042,817.428530,412.778500,1.0,1018.394040,12.000000,50.0,2351.000000,0.000000
max,-1.802700,0.0,16.389470,0.008170,63.203430,0.581700,12.903438,70.0,1.0,50.0,...,33.015210,0.716042,3140.576660,496.364990,1.0,1046.191890,17.107660,50.0,5477.000000,1.000000


In [7]:
selected_columns = set(val.columns.drop(['timestamp', 'attack']))
columns_to_remove = []
mins, maxes = val.min(), val.max()
for col in val.columns:
    if mins[col] == maxes[col]:
        selected_columns.remove(col)
selected_columns = sorted(list(selected_columns))
print(selected_columns)

['C01', 'C03', 'C04', 'C05', 'C06', 'C07', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C21', 'C23', 'C24', 'C25', 'C27', 'C28', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C37', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C47', 'C48', 'C50', 'C51', 'C53', 'C54', 'C56', 'C57', 'C59', 'C60', 'C61', 'C62', 'C64', 'C65', 'C66', 'C67', 'C68', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C80', 'C81', 'C83', 'C84', 'C86']


In [8]:
train_col = ['C01', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C20', 'C21', 'C23', 'C24', 'C25', 'C27', 'C28', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C37', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C50', 'C51', 'C53', 'C54', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C64', 'C65', 'C66', 'C67', 'C68', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C83', 'C84', 'C86']
print(set(train_col)-set(selected_columns))
print(set(selected_columns)-set(train_col))

{'C20', 'C58', 'C46', 'C08', 'C79'}
set()


In [15]:
val[['C20', 'C58', 'C46', 'C08', 'C79', 'attack']].describe()

,C20,C58,C46,C08,C79,attack
count,86400.000000,8.640000e+04,86400.0,86400.0,86400.000000,86400.000000
mean,27.796206,2.098959e+01,25.0,70.0,0.716042,0.010243
std,0.000000,3.552734e-15,0.0,0.0,0.000000,0.100689
min,27.796206,2.098959e+01,25.0,70.0,0.716042,0.000000
25%,27.796206,2.098959e+01,25.0,70.0,0.716042,0.000000
50%,27.796206,2.098959e+01,25.0,70.0,0.716042,0.000000
75%,27.796206,2.098959e+01,25.0,70.0,0.716042,0.000000
max,27.796206,2.098959e+01,25.0,70.0,0.716042,1.000000


In [13]:
val[['C20', 'C58', 'C46', 'C08', 'C79', 'attack']].corr(method='pearson')

,C20,C58,C46,C08,C79,attack
C20,NaN,NaN,NaN,NaN,NaN,NaN
C58,NaN,NaN,NaN,NaN,NaN,NaN
C46,NaN,NaN,NaN,NaN,NaN,NaN
C08,NaN,NaN,NaN,NaN,NaN,NaN
C79,NaN,NaN,NaN,NaN,NaN,NaN
attack,NaN,NaN,NaN,NaN,NaN,1.0


In [9]:
test_dirs = glob.glob('../datasets/test/*.csv')
test_datas = [pd.read_csv(path) for path in test_dirs]
integrated = pd.DataFrame(columns=test_datas[0].columns)
for data in test_datas:
    integrated = integrated.append(data)
integrated.describe()

,C01,C03,C04,C05,C06,C07,C12,C13,C15,C16,...,C74,C75,C76,C77,C78,C79,C80,C81,C83,C84
count,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,...,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000,274800.000000
mean,-2.690713,12.083211,-0.000026,8.297553,-0.312005,12.589180,14.710807,28.034373,27.714673,30.798443,...,296.441533,31.294590,48.790446,1.118301,29.375247,5.018883,296.407937,366.698528,993.200363,12.003853
std,0.568128,0.240035,0.001641,6.465559,0.481061,0.057858,3.409162,0.003675,0.772048,3.719800,...,415.221099,3.772151,1.484043,0.111363,0.367961,2.690479,415.208883,59.691689,31.067213,0.157915
min,-4.380000,11.781310,-0.011280,0.276180,-1.090400,0.000000,0.268550,28.019500,26.129270,0.000000,...,7.150280,0.352480,42.637700,-0.012820,28.269350,2.403980,7.150280,230.523010,954.549500,12.000000
25%,-3.129000,11.903380,-0.001090,3.747560,-0.609300,12.588700,12.849430,28.031660,27.042940,28.447560,...,26.018910,28.871150,47.581760,1.053770,29.148980,2.851530,26.018910,319.733830,966.991880,12.000000
50%,-2.766800,12.017820,0.000000,6.280520,-0.430300,12.588700,14.802550,28.034260,27.694240,30.406690,...,91.563020,30.992130,48.542535,1.118620,29.357690,3.344280,91.563020,364.800350,980.137510,12.000000
75%,-2.447800,12.368770,0.001010,13.833620,-0.243500,12.588740,16.542050,28.036860,28.402620,32.541800,...,319.974270,33.044430,49.917792,1.181180,29.580420,8.812420,319.974270,414.026310,1024.175660,12.000000
max,0.000000,25.239560,0.015480,34.913640,1.479200,18.000000,92.378230,28.054230,29.441470,75.197480,...,2228.758060,76.516720,57.889610,2.059330,31.027710,9.960400,2228.758060,499.782960,1047.185670,25.000000


In [10]:
selected_columns_t = set(integrated.columns.drop('timestamp'))
columns_to_remove = []
mins, maxes = integrated.min(), integrated.max()
for col in integrated.columns:
    if mins[col] == maxes[col]:
        selected_columns_t.remove(col)
selected_columns_t = sorted(list(selected_columns_t))
print(selected_columns_t)

['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C20', 'C21', 'C23', 'C24', 'C25', 'C27', 'C28', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C37', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C47', 'C48', 'C50', 'C51', 'C53', 'C54', 'C56', 'C57', 'C58', 'C59', 'C60', 'C61', 'C62', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C80', 'C81', 'C83', 'C84', 'C86']


In [11]:
print(set(train_col)-set(selected_columns_t))
print(set(selected_columns_t)-set(train_col))

{'C46', 'C08'}
{'C69', 'C02'}
